In [1]:
import folium

center = (37.301104, -119.721280) # center of cali
m = folium.Map(location=center, zoom_start=6)

folium.raster_layers.TileLayer(
    tiles="https://gis.conservation.ca.gov/server/rest/services/CGS/FaultActivityMapCA/MapServer/tile/{z}/{y}/{x}",
    name="CGS Faults (toggle)",
    attr="California Geological Survey",
    overlay=True, control=True,
    show=True,          # show fault lines
    max_zoom=14, opacity=0.9
).add_to(m)

### Data Prep: Population Data

In [2]:
import pandas as pd
import geopandas as gpd

df = pd.read_csv("datasets/MCNA_-_Population_Points_with_T_D_Standards.csv")

/tmp/ipykernel_6891/3146788476.py:4: DtypeWarning: Columns (91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("datasets/MCNA_-_Population_Points_with_T_D_Standards.csv")


In [3]:
df.drop(columns=[
    "CENSUS_TRACT",
    "CENSUS_BLOCK_GROUP",
    "ALL_PLAN",
    "County_Size_Type",
    "Loc_Code_Gcode"
], axis=1, inplace=True)

for col in df.columns:
    if "DT" in col or "TT" in col:
        df.drop(col, axis=1, inplace=True)

In [4]:
import geopandas

geometry = geopandas.points_from_xy(df.LONGITUDE, df.LATITUDE)
geo_df = geopandas.GeoDataFrame(
    df, geometry=geometry, crs="EPSG:4326"
)

# clear N/A values
geo_df = geo_df.dropna(subset=["LATITUDE", "LONGITUDE"])

In [5]:
from folium.plugins import HeatMap
heat_data = [[pt.y, pt.x] for pt in geo_df.geometry]

HeatMap(
    heat_data,
    name="Population heat",
    radius=12,
    blur=18,
    min_opacity=0.25,
    max_zoom=12,
    show=True,          # make it visible initially
    control=True
).add_to(m)

In [6]:
folium.LayerControl().add_to(m)

In [7]:
m